In [22]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn as nn
from torch.autograd import Variable
import os,sys
from PIL import Image
import tensorflow as tf
from IPython.display import clear_output
import tensorflow_datasets as tfds
from glob import glob
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
SEED = 42

dataset_path = "training/" 
training_data = "images/"

# Image size that we are going to use
IMG_SIZE = 400
# Our images are RGB (3 channels)
N_CHANNELS = 3
# Scene Parsing has 150 classes + `not labeled`
N_CLASSES = 2

In [45]:
TRAINSET_SIZE = len(glob(dataset_path + training_data + "*.png"))
print(f"The Training Dataset contains {TRAINSET_SIZE} images.")


The Training Dataset contains 100 images.


In [56]:
def parse_image(img_path: str) -> dict:
    """Load an image and its annotation (mask) and returning
    a dictionary.

    Parameters
    ----------
    img_path : str
        Image (not the mask) location.

    Returns
    -------
    dict
        Dictionary mapping an image and its annotation.
    """
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.uint8)

    # For one Image path:
    # .../trainset/images/training/ADE_train_00000001.jpg
    # Its corresponding annotation path is:
    # .../trainset/annotations/training/ADE_train_00000001.png
    mask_path = tf.strings.regex_replace(img_path, "image", "groundtrouth")
    mask = tf.io.read_file(mask_path)
    # The masks contain a class index for each pixels
    mask = tf.image.decode_png(mask, channels=1)
    # In scene parsing, "not labeled" = 255
    # But it will mess up with our N_CLASS = 150
    # Since 255 means the 255th class
    # Which doesn't exist
    mask = tf.where(mask == 255, np.dtype('uint8').type(0), mask)
    # Note that we have to convert the new value (0)
    # With the same dtype than the tensor itself

    return {'image': image, 'groundtrouth': mask}

In [57]:
train_dataset = tf.data.Dataset.list_files(dataset_path + training_data + "*.png", seed=SEED)
train_dataset = train_dataset.map(parse_image)
print(len(train_dataset))

100


In [69]:
def load_image(datapoint: dict) -> tuple:
    """Apply some transformations to an input dictionary
    containing a train image and its annotation.

    Notes
    -----
    An annotation is a regular  channel image.
    If a transformation such as rotation is applied to the image,
    the same transformation has to be applied on the annotation also.

    Parameters
    ----------
    datapoint : dict
        A dict containing an image and its annotation.

    Returns
    -------
    tuple
        A modified image and its annotation.
    """
    input_image = tf.image.resize(datapoint['image'], (400, 400))
    input_mask = tf.image.resize(datapoint['groundtrouth'], (400, 400))

    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask


In [70]:
train_dataset = train_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

In [74]:
def display_sample(display_list):
    """Show side-by-side an input image,
    the ground truth and the prediction.
    """
    plt.figure(figsize=(18, 18))

    title = ['Input Image', 'True Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [75]:
for image, mask in train_dataset.take(1):
    sample_image, sample_mask = image[0], mask[0]
    
display([sample_image, sample_mask])


2021-11-18 19:55:06.181152: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: training/groundtrouths/satImage_047.png; No such file or directory
2021-11-18 19:55:06.181212: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: training/groundtrouths/satImage_040.png; No such file or directory
2021-11-18 19:55:06.188322: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: training/groundtrouths/satImage_003.png; No such file or directory
2021-11-18 19:55:06.188383: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: training/groundtrouths/satImage_039.png; No such file or directory
2021-11-18 19:55:06.191055: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:116 : NOT_FOUND: training/groundtrouths/satImage_075.png; No suc

NotFoundError: training/groundtrouths/satImage_003.png; No such file or directory
	 [[{{node ReadFile_1}}]] [Op:IteratorGetNext]

In [3]:
def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1
    return input_image, input_mask


In [7]:
#def load_image(datapoint):
    #input_images = tf.image.resize(datapoint['images'], (400, 400))
    #input_groundtruth = tf.image.resize(datapoint['groundtruth'], (400, 400))

    #input_images, input_groundtruth = normalize(input_images, input_groundtruth)

    #return input_images, input_groundtruth

def load_image(infilename):
    data = mpimg.imread(infilename)
    data= tf.Variable(data)
    data= tf.convert_to_tensor(labels, dtype=tf.int32)

    return data

def load_image(infilename):
    data= tf.read_file(infilename)
    return data



In [ ]:
#imgs, gt_imgs = load_image(data_dir)

In [18]:
# Loaded a set of images
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(20, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

n = 10 # Only use 10 images for training
#imgs=normalize(imgs)



Loading 20 images
satImage_052.png
Loading 20 images
satImage_052.png


In [13]:
imgs=tf.convert_to_tensor(imgs)
for i in imgs.take(2):
    sample_image= imgs[0]
    display([sample_image])

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'take'